## LIBRARIES

In [1]:
!pip install contractions
!pip install gensim=="4.1.2"
!pip install sentence-transformers
!pip install scikit-learn-extra

     |████████████████████████████████| 106 kB 7.0 MB/s 
     |████████████████████████████████| 284 kB 38.6 MB/s 
     |████████████████████████████████| 24.1 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 3.8 MB 11.0 MB/s 
     |████████████████████████████████| 1.2 MB 42.3 MB/s 
     |████████████████████████████████| 67 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 64.2 MB/s 
     |████████████████████████████████| 895 kB 54.7 MB/s 
     |████████████████████████████████| 6.5 MB 50.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=29d98a348d57f18b6830f3b331393836e81e4f24eee10d12006beb5299e8f639
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391c

In [2]:
import string
from nltk.util import ngrams
from tqdm import tqdm
import csv
import re
import string
import operator
from datetime import datetime
import glob
import unicodedata
import re
import contractions
from contractions import contractions_dict
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import pandas as pd
from tqdm import tqdm
from yellowbrick.text import FreqDistVisualizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import spacy
import matplotlib.pyplot as plt
from collections import Counter
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.nmf import Nmf
import pickle
from sentence_transformers import SentenceTransformer
from sklearn_extra.cluster import KMedoids
import keras
from keras.layers import Input, Dense
from keras.models import Model
import os
from keras import backend as K
import random as rn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
# import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
import tensorflow as tf
import joblib
import warnings
warnings.filterwarnings('ignore', category=Warning)

os.environ['PYTHONHASHSEED']='0'
os.environ['CUDA_VISIBLE_DEVICES']=''
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["TOKENIZERS_PARALLELISM"]="false"

In [3]:
nltk.download('wordnet')
nltk.download('stopwords')  # GUARDARE PIU AVANZATE -> NN
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/

'0 - RESULTS'   GEA    IL    OLD_ProdLDA      PIPELINE_SPLIT.ipynb   TEST.ipynb
 data_dir       GEAC   IM    OTHER	      ProdLDA		     WORD2VEC
 DeepLog        HW2    IoT   PIPE	      T2V
 DSL	        HW3    MML   PIPELINE.ipynb   Tesi


In [7]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/

/content/gdrive/My Drive/Colab Notebooks


In [8]:
pathname="/content/gdrive/MyDrive/Colab Notebooks/"

## INPUT VARIABLES


In [32]:
var=None
while var not in ["Spark","HDFS"]:
    var = input(f"Inser dataset name: ")
if var=="Spark":
    name="Spark"
    model=None
else:
    name="HDFS"
    model=None

var=None
flag=True
path=None

while flag:
  var = input(f"Inset path input file: ")
  path=[var]
  var2= input("Change path? [y/n]")
  if var2 in ["no","n"]:
    flag=False  
print(f"COMPLETE PATH: {pathname+var}")
Origin=True


var=None
while var not in ["yes","no","y","n"]:
    var = input(f"Process with ORIGIN: ")

if var in ["yes","y"]:
    Origin=True
    folder = "ORIGIN+MESSAGE"
else:
    Origin=False
    folder = "MESSAGE"

Inser dataset name: Spark
Inset path input file: /content/gdrive/MyDrive/Colab Notebooks/PIPE/INPUT/Spark_input.log
Change path? [y/n]n
COMPLETE PATH: /content/gdrive/MyDrive/Colab Notebooks//content/gdrive/MyDrive/Colab Notebooks/PIPE/INPUT/Spark_input.log
Process with ORIGIN: n


/content/gdrive/MyDrive/Colab Notebooks/PIPE/INPUT/Spark_input.log

## FUNCTIONS

### DATA MANIPULATION

#### PROCESSING FUNCTIONS

In [33]:
def ImportFiles():
    print("\tIMPORT FILES")
    filepath = f"C:/Users/david/Desktop/Tesi/Dataset/{name}/{name}/*.log"
    paths = glob.glob(filepath)
    # print(len(paths))
    if len(paths) == 0:
        filepath = f"C:/Users/david/Desktop/Tesi/Dataset/{name}/{name}/*/*.log"
        paths = glob.glob(filepath)
        # print(len(paths))
    # for i in paths:
    #     print(i)
    return paths

In [34]:
def ReadFile(paths, index):
    #print("\tREAD FILE")
    path0 = paths[index]  # PER ORA CONSIDERO SOLO UN FILE DI LOG
    print(f"\t{path0}")
    file = open(path0, 'r')
    lines = file.read().splitlines()
    file.close()
    i = 0
    outread = []
    for ln, line in enumerate(lines):
        if (len(line) > 0):
            if (line.startswith((' ', "\t"))) or (line.startswith(('STARTUP_MSG', "/*", "**", "SHUTDOWN_MSG"))):
                None
            else:
                outread.append(line)
                i += 1
    return outread

In [35]:
def MakeDataframe(outread):
    #print("\tMAKE PANDAS DATAFRAME")
    records = []
    for i, line in enumerate(outread):
        tmp = []
        counter = 0
        substring = ""
        firstchar = True
        errorline = False
        for char in line:
            if firstchar:
                firstchar = False
                if char.isalpha():  # FIRST CHAR AT EACH LINE IS NUMBER OF THE DATE
                    print(f"FIRST CHAR ERROR LINE: {i} - {char}")
                    errorline = True
                    break
            if char == " " and counter < 4:
                tmp.append(substring)
                substring = ""
                counter += 1
            else:
                substring += char
        if counter >= 4:
            tmp.append(substring)
        if errorline:
            splits = line.split(":")
            print(len(splits))
            orig = splits[0]
            if len(splits) > 1:
                ms = ":".join(splits[1:])
            else:
                ms = orig
                orig = ""
            tmp = [" ", " ", " ", orig, ms]
            print(f"ERROR LINE: {i} - {tmp}\n")
        if len(tmp) != 5:
            print("ERRORE")
            print(line)
            print(f"{tmp}\n")
        records.append(tmp)
    df = pd.DataFrame(records, columns=["DATE", "TIMESTAMP", "TYPE", "ORIGIN", "MESSAGE"])
    if Origin:
        #print("\t\tWITH ORIGIN")
        df2 = df.loc[:, ["ORIGIN", "MESSAGE"]]
        df2["STRING"] = df2["ORIGIN"] + " " + df2["MESSAGE"]  # PROVARE SOLO MESSAGE -> NO DIFFERENCES(TO_SAVE)
        df2 = df2.loc[:, ["STRING"]]
    else:
        #print("\t\tWITHOUT ORIGIN")
        df2 = df.loc[:, ["MESSAGE"]]
    # print(df2.head())
    # origins = df2["ORIGIN"].tolist()
    # messages=df2["MESSAGE"].tolist()
    # input_preproc=[]
    # for o,m in zip(origins, messages):
    #     row=o+" "+m
    #     input_preproc.append(row)
    input_preproc = df2.values.tolist()
    return input_preproc

In [36]:
def RemoveAccent(input_preproc):
    def remove_accented_chars(text):
        text = (unicodedata
                .normalize('NFKD', text)
                .encode('ascii', 'ignore')
                .decode('utf-8', 'ignore'))
        return text
    print("\tREMOVE POSSIBLE ACCENTED CHARACTERS")
    no_accent = []
    for i, el in (enumerate(input_preproc)):
        tmp = remove_accented_chars(el[0])
        # if el!=tmp:
        #     print("MODIFIED")
        #     print(f"{i} - {el[0]}")
        #     print(f"{i} - {tmp}\n")
        no_accent.append(tmp)  # LOWERCASE, CONSIDERAZIONE CAMEL CAPITALIZER
    return no_accent

In [37]:
def RemoveSpecial(ccs):
    set_upp = set()
    def remove_special_characters(text, i):  # TRA 2 /
        brackets = False
        first_len = len(text)
        text = re.sub("[\(\[].*?[\)\]]", " ", text)  # PARANTESI TONDE E QUADRE
        second_len = len(text)
        if first_len != second_len:
            brackets = True
        build = []
        possible_path = []
        SOURCE = True
        for el in text.split(" "):  # RIMOZIONE PATH
            if not SOURCE:  # PRIMO ELEMENTO SOURCE
                if "/" in el:
                    tmp_split = el.split("/")
                    str1 = "".join([c for c in tmp_split[0] if c not in string.punctuation])
                    str2 = "".join([c for c in tmp_split[1] if c not in string.punctuation])
                    if (len(tmp_split) == 2 and (len(str1) == 1 or len(str2) == 1)):
                        if len(str1) == 1:
                            build.append(tmp_split[1])
                        else:
                            build.append(tmp_split[0])
                    else:
                        possible_path.append(el)
                else:
                    build.append(el)
            else:
                build.append(el)
            SOURCE = False
        text = " ".join(build)
        text = re.sub(' +', ' ', text)
        build = []
        word_with_digit = []
        SOURCE = True
        for el in text.split(" "):
            if not SOURCE:
                array = re.findall(r'[0-9]+', el)
                if len(array) == 0:
                    build.append(el)
                else:
                    word_with_digit.append(el)
            else:
                build.append(el)
            SOURCE = False
        text = " ".join(build)
        text = re.sub(r'[@|;|$]+', ' ', text)  # SPLIT BY TOKENS
        r1 = re.compile(r"(([\w]+(:|=))+(-|)([\w]+))")  # ([0-9]+|[A-Za-z-]+[0-9]+))")  # npid=261268536, #pid=null
        build = []
        possible_word_eq_dp_num = []
        for el in text.split(" "):
            if r1.match(el):
                possible_word_eq_dp_num.append(el)
            else:
                build.append(el)
        text = " ".join(build)

        # text = re.sub('[^\'a-zA-Z\s]', ' ', text) #TENGO SOLO CARATTERI
        # r4 = re.compile(r"'[a-zA-Z]+'")
        # output = []
        # word_with_apex=[]
        # for word in text.split():
        #     if r4.match(word):
        #         word_with_apex.append(word)
        #         output.append(word.replace("'", ""))
        #     else:
        #         output.append(word)
        # text=" ".join(output)
        r4 = re.compile(r"'[.\w]+'")
        output = []
        word_with_apex = []
        for word in text.split():
            if r4.match(word):
                if len(word) < 20:
                    word_with_apex.append(word)
                    output.append(word.replace("'", ""))
            else:
                output.append(word)
        text = " ".join(output)
        text = re.sub('[^\'a-zA-Z\s]', ' ', text)  # TENGO SOLO CARATTERI
        text = re.sub(r'[\r|\n|\r\n_|\r\n/]+', ' ', text)  # TOLGO POTENZIALI SIMBOLI RIMASTI
        text = text.replace("[", "")
        text = text.replace("]", "")
        text = re.sub(' +', ' ', text.strip())  # TOLGO SPAZI AGGIUNTIVI
        res = []
        split_upper = []
        for el in text.split(" "):
            res.append(el)
            prevUpp = 0
            uppstring = ""
            for char in el:
                if char.isupper():
                    prevUpp += 1
                    uppstring += char
                else:
                    if prevUpp >= 3:
                        uppstring = uppstring[0:len(uppstring) - 1]
                        tmp = el
                        lowstirng = tmp.replace(uppstring, "")
                        res.remove(el)
                        res.append(uppstring)
                        res.append(lowstirng)
                        split_upper.append(uppstring)
                        split_upper.append(lowstirng)
                        set_upp.add(el)
                    prevUpp = 0
                    uppstring = ""
        out = " ".join(res)
        return out, possible_path, possible_word_eq_dp_num, word_with_digit, brackets, word_with_apex, split_upper
    print("\tREMOVE SPECIAL")
    no_special = []
    for i, el in (enumerate(ccs)):
        tmp, path, wen, wwd, br, wwa, su = remove_special_characters(el, i)
        #if len(su) > 0:  # and len(path)>0  and len(wwd)>0:
            #print(f"{i} - {el}")
            #print(f"{i} - {tmp}")
            #     print(f"{i} - PATHS: {path}")
            #     print(f"{i} - W=:N: {wen}")
            #     print(f"{i} - WWA: {wwa}")
            #print(f"{i} - SU: {su}")
        #     print(f"{i} - WWD: {wwd}\n")
        no_special.append(tmp)
    return no_special

In [38]:
def CamelCase(no_accent):
    tot_words = set()
    def camel_case_split(str):
        words = [[str[0]]]
        for c in str[1:]:
            if words[-1][-1].islower() and c.isupper():
                words.append(list(c))
            else:
                words[-1].append(c)
        inner_tmp = [''.join(word) for word in words]
        for ww in inner_tmp:
            tot_words.add(ww)
        out_tmp = ' '.join(inner_tmp)
        return out_tmp
    print("\tCAMEL CASE SPLIT")
    ccs = []
    for i, el in (enumerate(no_accent)):
        tmp = camel_case_split(el)
        tmp = tmp.lower()
        #if i < 69:
            #if el != tmp:
                #print(f"{i} - {el}")
                #print(f"{i} - {tmp}")
        #     if el!=tmp:
        #         print("MODIFIED!\n")
        #     else:
        #         print("\n")
        ccs.append(tmp)
    return ccs

In [39]:
def CorrectCamel(normalized):
    mergedwords = set()  ############################################################################### TODO A PRIORI
    AllWord = set()
    for el in (normalized):  # ALL DATA
        for wrd in el.split(" "):
            AllWord.add(wrd)
    #print(len(AllWord))
    def create_merged_words(str):
        List_w = str.split(" ")
        List_w = [x for x in List_w if len(x) > 0]
        j = 0
        for i in range(len(List_w)):
            if j >= len(List_w):
                break
            if (j + 1) < len(List_w):
                tmp_w = List_w[j] + List_w[j + 1]
                if tmp_w in AllWord:
                    if "block" not in tmp_w:
                        mergedwords.add(tmp_w)
                    j += 2
                else:
                    j += 1
            else:
                j += 1
    def wrong_camel_case_split(str):
        List_w = [x for x in str.split(" ") if len(x) > 0]
        row_set = set(List_w)
        row_list = list(row_set)
        output = []
        j = 0
        flag = 0
        for i in range(len(List_w)):
            if j >= len(List_w):
                break
            if (j + 1) < len(List_w):
                tmp_w = List_w[j] + List_w[j + 1]
                if tmp_w in row_list or tmp_w in mergedwords:
                    flag = 1
                    output.append(tmp_w)
                    j += 2
                else:
                    output.append(List_w[j])
                    j += 1
            else:
                output.append(List_w[j])
                j += 1
        out = " ".join(output)
        return out, flag
    print("\tCONCATENATION WRONG CAMEL CASE")
    w_css = []
    for i, el in (enumerate(normalized)):
        create_merged_words(el)
    #print(f"MERGED WORDS: {mergedwords}")
    for i, el in (enumerate(normalized)):
        tmp, flag = wrong_camel_case_split(el)
        #if i < 69:
            #if flag:
                #print(f"{i} - {el}")
                #print(f"{i} - {tmp}")
        #     if flag:
        #         print("MODIFIED!\n")
        #     else:
        #         print("\n")
        w_css.append(tmp)
    return w_css

In [40]:
def ExpandContraction(w_css):
    def expand_contractions(text):
        expanded_words = []
        for word in text.split():
            fix_w = contractions.fix(word)
            expanded_words.append(fix_w)
        out_tmp = ' '.join(expanded_words)
        return out_tmp
    print("\tEXPAND POSSIBLE CONTRACTIONS")
    expand = []
    for i, el in (enumerate(w_css)):
        tmp = expand_contractions(el)
        #if el != tmp:
            #print("MODIFIED!\n")
            #print(f"{i} - {el}")
            #print(f"{i} - {tmp}\n")
        expand.append(tmp)
    return expand

In [41]:
def Tokenization(expand):
    print("\tTOKENIZATION")
    tokenize = []
    for i, el in (enumerate(expand)):
        tmp = word_tokenize(el)
        tokenize.append(tmp)
    return tokenize

In [42]:
def RemoveStopwords(tokenize):
    print("\tREMOVE POSSIBLE STOPWORDS")
    stop = stopwords.words('english')
    stop.append(name.lower())  # DATASET NAME
    stop.remove("not")
    stop.remove("no")
    stop.remove("nor")
    stop.remove("down")
    no_stop = []
    for i, el in (enumerate(tokenize)):
        tmp = []
        for x in el:
            if x not in stop:
                if x == "no" or x == "nor":
                    tmp.append("not")
                else:
                    tmp.append(x)

        # if i < 69:
        #     print(f"{i} - {el}")
        #     print(f"{i} - {tmp}")
        #     if len(el)!=len(tmp):
        #         print(f"MODIFIED! REMOVED: {[y for y in el if y not in tmp]}\n")
        #     else:
        #         print("\n")
        no_stop.append(tmp)
    return no_stop

In [43]:
def PosTagChunk(no_stop):
    print("\tPOS TAGGING & CHUNKING")
    tagg = []
    for i, el in (enumerate(no_stop)):
        tmp = pos_tag(el)
        # if i < 100:
        #     print(f"{i} - {el}")
        #     print(f"{i} - {tmp}\n")
        tagg.append(tmp)
    return tagg

In [44]:
def StemmLemm(tagg):  ################################################################################ TODO USER AWARE WORD WITH MULTIPLE TAGS
    print("\tLEMMING")
    admitted_tag = ["JJ", "JJR", "JJS", "NN", "NNS", "NNP", "NNPS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]
    stemm = []
    wl = WordNetLemmatizer()
    diz_word = {}
    wrong_word_set = set()
    word_ing = set()
    wrong_word_idx = []
    to_keep_wrong_tag = ["not", "filter", "namenode", "second", "timeout", "util", "down", "server", "driver"]
    def get_wordnet_pos(tokens, idx=-1):
        tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN,
                    "V": wordnet.VERB}  # , "R": wordnet.ADV} #EXTEND???
        tmp = []
        for token in tokens:
            # print(token)
            word = token[0]
            tag = token[1]
            if word.endswith("ing") and tag in admitted_tag and tag[0] != "V":
                word_ing.add((word, tag))
                tag = "VB"
            if tag[0] != "V" and word.endswith("ed"):
                wrong_word_set.add(word)
                if idx < 0:
                    wrong_word_idx.append(idx)
                tag = "VB"
            if tag in admitted_tag or word in to_keep_wrong_tag:
                # diz_word[word]=tag
                tmp.append((word, tag_dict.get(tag[0], wordnet.NOUN)))
        # print(tmp)
        return tmp
    word_3 = set()
    to_keep = ["max", "new", "net", "msg", "bad", "os", "ipc", "not", "eof", "dfs", 'io', "up", "map", "use", "bp",
               "nodename", "nn", "no", "src", "dst", "end", "rdd", "server"]
    avoid_lemm_error = ["dest", "os", "not", 'io', "server"]
    my_lemma = {"rescanning": "rescan", "rescanned": "rescan", "dst": "dest", "remoting": "remote",
                "deprecation": "deprecate"}
    # java, io, eof, reset, socket, file, org, datanode, user, ipc, impl, storage, bpid, block, bytes, nn, src, dest, write, id, finalize, interval, total, fsdataset, replicas
    # TODO USER INPUT
    for i, el in (enumerate(tagg)):
        for w, t in el:
            if w not in diz_word.keys():
                diz_word[w] = set()
                t_wn = get_wordnet_pos([(w, t)])
                if len(t_wn) > 0:
                    diz_word[w].add(t_wn[0][1])
                if len(diz_word[w]) == 0:
                    print(f"\t\t{w} NOT added since {t} NOT admitted")
                    var = ""
                    while var not in ["y", "n"]:
                        var = input(f"\t\tDo you want to change the TAG for {t} (prev {t}): ")
                        print("\t\tYou entered: " + var)
                        if var in ["y", "n"]:
                            if var == "y":
                                var2 = ''
                                while var2 not in ['a', 'v', 'n']:
                                    var2 = input(f"\t\tChoose tag between 'a','v' and 'n':")
                                    if var2 in ['a', 'b', 'n']:
                                        t_wn = get_wordnet_pos([(w, var2)])
                                        if len(t_wn) > 0:
                                            diz_word[w].add(t_wn[0][1])
                            else:
                                print(f"\t\tElement Discarded")
                        else:
                            print(f'\t\tInput not acceptable, choose between: {["y", "n"]}')
                    print("\n")
            else:
                t_wn = get_wordnet_pos([(w, t)])
                if len(t_wn) > 0:
                    diz_word[w].add(t_wn[0][1])
    word_tags = {}
    tot_err = 0
    for k, v in diz_word.items():
        v = list(v)
        if len(v) > 1:
            tot_err += 1

    print(f"\t\tThere are {tot_err} words with more than one tag")

    tag_wn = {"a": "ADJECTIVE", "n": "NOUN", "v": "VERB", "d": "DISCARDA TAG AGGREGATION ANALYSIS FOR THAT WORD"}
    to_pop = []
    for k, v in diz_word.items():
        if len(v) == 0:
            to_pop.append(k)
        else:
            v = list(v)
            if len(v) > 1:
                kb = ""
                for char in k:
                    kb = kb + "\u0332" + char + "\u0332"
                print(f"\t\tConflict for: {kb}, TAGS found: {v}")
                set_row = set()
                for orig in no_special:
                    ispresent = False
                    for wrd in orig.split(" "):
                        if wrd.lower() == k:
                            ispresent = True
                    if ispresent:
                        tmp = []
                        for wrd in orig.split(" "):
                            if wrd.lower() == k:
                                tmpwrd = ""
                                for char in wrd:
                                    tmpwrd = tmpwrd + "\u0332" + char + "\u0332"
                                tmp.append(tmpwrd)
                            else:
                                tmp.append(wrd)
                        toadd = " ".join(tmp)
                        set_row.add(toadd)
                var = None
                print(f"\t\tHere the first 5 row to decide the TAG")
                it_tmp = 0
                for el in set_row:
                    if it_tmp < 5:
                        print(f"{it_tmp} - {el}")
                    else:
                        break
                    it_tmp += 1
                # v.append("d")
                while var not in v:
                    guideline = {}
                    for el in v:
                        if el == "a":
                            guideline[el] = "ADJECTIVE"
                        if el == "n":
                            guideline[el] = "NOUN"
                        if el == "v":
                            guideline[el] = "VERB"
                    guideline["d"] = "DISCARDA TAG AGGREGATION ANALYSIS FOR THAT WORD"
                    var = input(f"\t\tPlease choose the desidered TAG between {v} {guideline}: ")
                    print("\t\tYou entered: " + var)
                    if var not in v:
                        if var in tag_wn.keys():
                            var2 = input(f"\t\tYou have inserted another acceptable TAG: {var}. Do you confirm it? [y/n]")
                            if var2 == "y":
                                word_tags[k] = var
                                print(f"\t\tModification Accepted!\n")
                                break
                            if var2 == "n":
                                print(f"\t\tModification Discarded!")
                        else:
                            print(f"\t\tInput not acceptable, choose between: {v}")
                    else:
                        word_tags[k] = var
                        print(f"\t\tInput Valid!\n")
            else:
                word_tags[k] = v
    for el in to_pop:
        diz_word.pop(el, None)
    #print("Start Lemmization")

    for i, el in (enumerate(tagg)):
        tmp = []
        res = get_wordnet_pos(el, i)
        for w in res:
            if len(w[0]) < 4 and w[0] not in to_keep:
                word_3.add(w[0])
        for w1, w2 in zip(el, res):
            if w2[0] in avoid_lemm_error:
                tmp.append(w2[0])
            else:
                if (w2[1] == "v" and len(w2[0]) > 2) or (w2[0] in to_keep) or (len(w2[0]) > 3):
                    if w2[0] in list(my_lemma.keys()):
                        tmp.append(my_lemma[w2[0]])
                    else:
                        if word_tags[w2[0]] != "d":
                            tmp.append(wl.lemmatize(w2[0], word_tags[w2[0]][0]))
                        else:
                            tmp.append(wl.lemmatize(w2[0], w2[1]))
        #if el != tmp:
         #   print(f"{i} - {el}")
          #  print(f"{i} - {tmp}")
           # print("MODIFIED\n")
        #else:
            #print("\n")
        stemm.append(tmp)
    #print(word_3)
    #print(wrong_word_set)
    #print(wrong_word_idx)
    #print(word_ing)
    return stemm, diz_word

In [45]:
def Regex(shorty):
    print("\tREGEX")
    r1r = re.compile(r"([bcdfghjklmnpqrstvwxz]*[aeiou]{3}[bcdfghjklmnpqrstvwxz]*)+")
    r2r = re.compile(r"([aeiou]*[bcdfghjklmnpqrstvwxz]{4}[aeiou]*)+")
    strange_w = set()
    r4 = set()
    r5 = set()
    to_keep = ["queue", "http"]
    def cleaning(text):
        output = []
        for wrd in text:
            if r1r.match(wrd) and len(wrd) < 6 and wrd not in to_keep:
                strange_w.add(wrd)
                r5.add(wrd)
            else:
                if r2r.match(wrd) and len(wrd) < 6 and wrd not in to_keep:
                    # print(wrd)
                    strange_w.add(wrd)
                    r4.add(wrd)
                else:
                    output.append(wrd)
        return output
    final = []
    for i, el in (enumerate(shorty)):  # ngrs
        tmp = cleaning(el)
       # if el != tmp:
           # print(f"{i} - {el}")
            #print(f"{i} - {tmp}")
          #  diff = [w for w in el if w not in tmp]
           # print(f"{i} - {diff}")
            #print("MODIFIED!\n")
        final.append(tmp)
    #print(f"\tStrange Words: {strange_w}")
    #print(f"{len(r4), len(r5)}")
    return final

In [46]:
def RemoveFinalS(input):
    TotWord = []
    for el in (enumerate(input)):  # ALL DATA
        for wrd in el[1]:
            TotWord.append(wrd)
    tot_w = list(set(TotWord))
    #print(len(tot_w), len(TotWord))
    new_row_set = []
    new_final = []
    count_modiication = 0
    for row in input:
        new_row = []
        for word in row:
            word_trunc = ""
            last_char = ""
            for i, char in enumerate(word):
                if i < len(word) - 1:
                    word_trunc += char
                else:
                    last_char = char
            if word_trunc in tot_w and last_char == "s":
                new_row.append(word_trunc)
            else:
                new_row.append(word)
        if row != new_row:
            count_modiication += 1
        if row != new_row and new_row not in new_row_set:
            #print("MODIFIED!")
           # print(row)
           # print(new_row)
            new_row_set.append(new_row)
        new_final.append(new_row)
    #print(len(input), (len(new_final)), count_modiication)
    return new_final

In [47]:
def NGRAMS(final, no_mono):
    print("\tMAKE NGRAMS")
    ngrs = []
    total_bigrams = set()  # NON NECESSARIO FARLO A PRIORI, OGNI RIGA SI BASA PER LE CORREZIONE SUI BIGRAMMI DELLE RIGHE PRECEDENTI
    total_trigrams = set()
    for i, el in enumerate(final):
        #print(el)
        tmp1_t = list(map(" ".join, list(ngrams(el, 1))))
        tmp1 = [el for el in tmp1_t if el not in no_mono]
        tmp2 = list(map(" ".join, list(ngrams(el, 2))))
        tmp3 = list(map(" ".join, list(ngrams(el, 3))))
        #print(tmp2)
        #print(tmp3)
        tmp2_3 = []
        for idx, ngrams_list in enumerate([tmp2, tmp3]):  # PRIMA BI E POI TRIGRAMMI
            inner_tmp = []
            for concat_words in ngrams_list:
                inner_words = concat_words.split(" ")
                single_words = set(inner_words)
                if len(single_words) == len(
                        inner_words):  # NON SONO PRESENTI ALMENO 2 PAROLE UGUALI -> SOLO PAROLE DISTINTE
                    if idx == 0:  # SE BIGRAM
                        # word1 = inner_words[0] + " " + inner_words[1]
                        word_reversed = inner_words[1] + " " + inner_words[0]
                        if word_reversed in total_bigrams:
                            #print(f"{i} - REVERSE ALREADY EXISTS: {word_reversed} of {concat_words}")
                            inner_tmp.append(word_reversed)
                            total_bigrams.add(word_reversed)
                        else:
                            inner_tmp.append(concat_words)
                            total_bigrams.add(concat_words)
                    else:
                        to_add = concat_words
                        inner_tmp.append(to_add)
                        total_trigrams.add(to_add)
                else:
                    None
                    #print(f"{i} - NOT ADDED: {concat_words}")
            tmp2_3 += inner_tmp
        tmp4 = tmp1 + tmp2_3  # tmp2 + tmp3 + tmp1
        tmp = tmp4
        #print(f"{i} - {el}")
        #print(f"{i} - {tmp}\n")
        ngrs.append(tmp)
    return ngrs

#### RUNNING FUNCTIONS

In [48]:
def Data():
  start_time = datetime.now()
  outread = ReadFile(path, 0)
  input_preproc = MakeDataframe(outread)
  if Origin:
      f=open(f"PIPE/{name}/{folder}/{name}_Input_ORIGIN+MESSAGE.txt","w")
  else:
      f=open(f"PIPE/{name}/{folder}/{name}_Input_MESSAGE.txt","w")
  for el in input_preproc:
      print(el[0], file=f)
  f.close()
  no_accent = RemoveAccent(input_preproc)
  no_special = RemoveSpecial(no_accent)
  tmp = no_special
  tmp_u = np.unique(np.array(tmp))
  #print(len(tmp), len(tmp_u))
  # %%
  if Origin:
      f = open(f"PIPE/{name}/{folder}/{name}_Clean_ORIGIN+MESSAGE.txt", "w")
  else:
      f = open(f"PIPE/{name}/{folder}/{name}_Clean_MESSAGE.txt", "w")
  for el in tmp_u:
      print(el, file=f)
  f.close()
  # %%
  ccs = CamelCase(no_special)
  # %%
  w_css = CorrectCamel(ccs)
  # %%
  expand = ExpandContraction(w_css)
  tokenize = Tokenization(expand)
  no_stop = RemoveStopwords(tokenize)
  # %%
  tmp = np.array(no_stop)
  tmpu, tmpind = np.unique(tmp, return_index=True)
  no_stop = tmpu.tolist()
  print(f"\tBEFORE: {len(tmp)}, AFTER: {len(no_stop)}, DIFF: {len(tmp) - len(no_stop)}, %: {100 * ((len(tmp) - len(no_stop)) / len(tmp))}%")
  tagg = PosTagChunk(no_stop)
  stemm2, diz_word = StemmLemm(tagg)
  final2 = Regex(stemm2)
  new_final2 = RemoveFinalS(final2)
  # %%
  nf2 = [" ".join(el) for el in new_final2]
  # %%
  nf = CorrectCamel(nf2)
  new_final2 = [el.split(" ") for el in nf]
  new_final = new_final2
  new_final = np.array(new_final)
  new_final_unique_max_f1 = []
  for row in new_final:
      r = np.array(row)
      ru = np.unique(r)
      new_final_unique_max_f1.append(ru)
  new_final_unique_max_f1 = np.array(new_final_unique_max_f1)
  # %%
  TotWordfinal_unique_max_f1 = []
  for el in (enumerate(new_final_unique_max_f1)):  # ALL DATA
      for wrd in el[1]:
          TotWordfinal_unique_max_f1.append(wrd)
  TotWordfinal_unique_max_f1 = np.array(TotWordfinal_unique_max_f1)
  TotWord_set = set(TotWordfinal_unique_max_f1)
  # %%
  diz_word_doc2 = {}
  for word in TotWord_set:
      diz_word_doc2[word] = 0
  for word in TotWord_set:
      for row in new_final_unique_max_f1:
          if word in row:
              diz_word_doc2[word] += 1
  sorted_d = dict(sorted(diz_word_doc2.items(), key=operator.itemgetter(1), reverse=True))
  keys = list(sorted_d.keys())
  values = list(sorted_d.values())
  # %%
  iter = 0
  for k, v in zip(keys, values):
      #print(iter, k, v)
      iter += 1
  # %%
  corpus = []
  for row in new_final_unique_max_f1:
      r = " ".join(row)
      #print(r)
      corpus.append(r)
  #print(len(corpus))
  # vectorizer = CountVectorizer(max_df=0.8)# min_df=2)#, ngram_range=(1, 3))
  # X = vectorizer.fit_transform(corpus)

  # features1 = vectorizer.get_feature_names()
  # diz_word_doc = {}
  # for word in features1:
  #     diz_word_doc[word] = 0
  # for word in features1:
  #     for row in new_final_unique_max_f1:
  #         if word in row:
  #             diz_word_doc[word] += 1
  # sorted_d = dict(sorted(diz_word_doc.items(), key=operator.itemgetter(1), reverse=True))
  # keys = list(sorted_d.keys())
  # values = list(sorted_d.values())
  # tot_len = len(new_final_unique_max_f1)
  # no_mono = []
  # for k, v in zip(keys, values):
  #     f = v / tot_len
  #     if f >= 0.2 and f < 0.8 and v!=1:
  #         no_mono.append(k)
  #     #print(k, v, f)
  f=open(f"PIPE/INPUT/{name}_drop.txt", "r")
  lines=f.readlines()
  to_drop=[]
  for el in lines:
    to_drop.append(el.rstrip("\n"))

  f=open(f"PIPE/INPUT/{name}_nomono.txt", "r")
  lines=f.readlines()
  no_mono=[]
  for el in lines:
    no_mono.append(el.rstrip("\n"))

  if len(no_mono)>0:
    print(f"\tWORDS TO TAKE ONLY THEIR BIGRAM/TRIGRAM INSIDE LOG ROWS: {no_mono}")

  in_data = []
  dropped_words = set()
  for row in (new_final):
      tmp = []
      for el in row:
          if el not in to_drop:
              tmp.append(el)
          else:
              dropped_words.add(el)
      in_data.append(tmp)
  if len(dropped_words)>0:
    print(f"\tDROPPED WORDS THAT ARE TO FREQUENT {dropped_words}")
  # %%
  input_data = NGRAMS(in_data, no_mono)
  # %%
  i = 0
  for el, el2 in zip(input_data, in_data):
      eligible = False
      eligible2 = False
      for w in el2:
          if w in dropped_words:
              eligible = True
          if w in no_mono:
              eligible2 = True
      #if eligible or eligible2:
          #print(i, el2, el)
      i += 1
  # %%
  end_time = datetime.now()
  #print('Duration: {}'.format(end_time - start_time))
  # %%
  if Origin:
      f = open(f"PIPE/{name}/{folder}/{name}_Input_Output_ORIGIN+MESSAGE.txt", "w")
      f2 = open(f"PIPE/{name}/{folder}/{name}_Raw_ORIGIN+MESSAGE.txt", "w")
  else:
      f = open(f"PIPE/{name}/{folder}/{name}_Input_Output_MESSAGE.txt", "w")
      f2 = open(f"PIPE/{name}/{folder}/{name}_Raw_MESSAGE.txt", "w")

  iter = 0
  for ind in tmpind:
      print(f"{iter} - {no_accent[ind]}", file=f)
      print(f"{no_accent[ind]}", file=f2)
      print(f"{iter} - {input_data[iter]}\n", file=f)  # SpaCy
      iter += 1
  f.close()
  f2.close()
  # %%
  #print("SAVE DATA")
  if Origin:
      fname = f"PIPE/{name}/{folder}/{name}_InputData_ORIGIN+MESSAGE.txt"
  else:
      fname = f"PIPE/{name}/{folder}/{name}_InputData_MESSAGE.txt"

  f = open(fname, "w")
  for el in input_data:
      print(el, file=f)
  #print("END SAVE DATA")
  f.close()

### TOPIC MODELS


#### PROCESSING FUNCTIONS

In [49]:
def get_topic_words(token_lists, labels, k=None, topk=20):
    if k is None:
        k = len(np.unique(labels))
    topics = ['' for _ in range(k)]
    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True),word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:topk])), word_counts))
    return topics
def get_coherence(model, token_list, measure='c_v', topk=20):
    topics = get_topic_words(token_list, model.cluster_model.labels_, topk=20)
    cm = CoherenceModel(topics=topics, texts = token_list, corpus=model.corpus, dictionary=model.dictionary, coherence = measure)
    return cm.get_coherence()

In [50]:
class Autoencoder:
    def __init__(self, latent_dim = 32, activation='relu', epochs=200, batch_size=128):
        self.latent_dim = latent_dim
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.autoencoder = None
        self.encoder = None
        self.decoder = None
        self.his = None
        self.Xtrain=None
        self.Xtest=None
    def _compile(self, input_dim):
        input_vec = Input(shape=(input_dim,))
        encoded = Dense(self.latent_dim, activation=self.activation)(input_vec)
        decoded = Dense(input_dim, activation=self.activation)(encoded)
        self.autoencoder = Model(input_vec, decoded)
        self.encoder = Model(input_vec, encoded)
        encoded_input = Input(shape=(self.latent_dim,))
        decoded_layer = self.autoencoder.layers[-1]
        self.decoder = Model(encoded_input, self.autoencoder.layers[-1](encoded_input))
        self.autoencoder.compile(optimizer='adam', loss=keras.losses.binary_crossentropy) #ADAM + MEAN ABS ERROR
    def fit(self, X):
        if not self.autoencoder:
            self._compile(X.shape[1])
        X_train, X_test = train_test_split(X, random_state=42)
        self.Xtrain=X_train
        self.Xtest=X_test
        self.his = self.autoencoder.fit(X_train, X_train, epochs=self.epochs, batch_size=self.batch_size, shuffle=False, validation_data=(X_test, X_test), verbose=0, workers=1)

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
#import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
#from Autoencoder import *
#from preprocess import *
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
from datetime import datetime

# define model object
class Topic_Model:
    def __init__(self, k=10, method='BERT', gamma=15, delta=15, epsilon=15, AE_latent_dim=32, AE_epochs=100, modelw2v=None, cluster_model=None, cluster_metric=None, scale=None, reduce=None, DEBUG=False):

        if method not in {'BERT', 'BERT_LDA', "BERT_LDA_W2V", "W2V", "W2V_LDA", "BERT_W2V", "BERT_NMF", "BERT_NMF_W2V", "W2V_NMF"}:
            raise Exception('Invalid method!')
        self.k = k
        self.dictionary = None
        self.corpus = None
        self.ldamodel = None
        self.nmfmodel = None
        self.modelw2v = modelw2v
        self.vec = {}
        self.gamma = gamma #15  # parameter for reletive importance of lda
        self.delta = delta
        self.epsilon = epsilon
        self.method = method
        self.AE = None
        self.AE_latent_dim=AE_latent_dim
        self.AE_epochs=AE_epochs
        self.ldavec=None
        self.nmfvec=None
        self.bertvec=None
        self.w2vvec=None
        self.cm=cluster_model
        self.scale=scale
        self.cluster_metric=cluster_metric
        self.cluster_model=None
        self.reduce=reduce
        self.DEBUG=DEBUG
        self.tw=None
        #self.cluster_model=None
        print(f"Method:{self.method}, Gamma:{self.gamma}, Delta:{self.delta}, Epsilon:{self.epsilon}, AE_latent_dim:{self.AE_latent_dim}, AE_epochs:{self.AE_epochs}, Cluster: {self.cm}, Metric: {self.cluster_metric}, Scale: {self.scale}, Reduce: {self.reduce}")

    def vectorize(self, sentences, token_lists, method=None):
        # Default method
        if method is None:
            method = self.method

        if self.dictionary is None:
          print("\tMAKE DICTIONARY")
          self.dictionary = corpora.Dictionary(token_lists)
        if method==self.method:
          print("\tMAKE CORPUS")
          self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        if method == 'LDA':
            print("LDA")
            print("\tLEN CORPUS",len(self.corpus))
            if self.ldamodel is None:
                print("\tTRAIN LDA MODEL")
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary, passes=10, iterations=100)

            def get_vec_lda(model, corpus, k):
                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob
                return vec_lda

            vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
            return vec

        elif method == 'NMF':
            print("NMF")
            print("\tLEN CORPUS",len(self.corpus))
            if self.nmfmodel is None:
                print("\tTRAIN NMF MODEL")
                self.nmfmodel = Nmf(self.corpus, num_topics=self.k, id2word=self.dictionary, passes=20, eval_every=100)

            def get_vec_nmf(model, corpus, k):
                n_doc = len(corpus)
                vec_nmf = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_nmf[i, topic] = prob

                return vec_nmf

            vec = get_vec_nmf(self.nmfmodel, self.corpus, self.k)
            return vec

        elif method == 'BERT':
            print("BERT")

            model = SentenceTransformer('all-mpnet-base-v2')#bert-base-nli-max-tokens')
            vec_bert = np.array(model.encode(sentences, show_progress_bar=False))

            return vec_bert
             
        elif method == 'BERT_LDA':
            vec_lda = self.vectorize(sentences, token_lists, method='LDA')
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            self.ldavec=vec_lda
            self.bertvec=vec_bert

            if self.gamma>0 and self.delta>0:
                vec_ldabert = np.c_[vec_lda * self.gamma, vec_bert*self.delta]
            elif self.gamma>0:
                vec_ldabert=vec_lda*self.gamma
            else:
                vec_ldabert=vec_bert*self.delta

            print(vec_lda.shape, vec_bert.shape, vec_ldabert.shape)

            return vec_ldabert

        elif method == 'BERT_NMF':
            vec_nmf = self.vectorize(sentences, token_lists, method='NMF')
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            self.nmfvec=vec_nmf
            self.bertvec=vec_bert

            vec_nmfbert = np.c_[vec_nmf * self.gamma, vec_bert*self.delta]

            print(vec_nmf.shape, vec_bert.shape, vec_nmfbert.shape)

            return vec_nmfbert
        
        elif method == "BERT_LDA_W2V":
            vec_lda = self.vectorize(sentences, token_lists, method='LDA')
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            vec_w2v = self.vectorize(sentences, token_lists, method='W2V')
            self.ldavec=vec_lda
            self.bertvec=vec_bert
            self.w2vvec=vec_w2v

            if self.gamma>0 and self.delta>0 and self.epsilon>0:
                vec_bertldaw2v = np.c_[vec_lda * self.gamma, vec_bert*self.delta, vec_w2v*self.epsilon]
            
            if self.gamma>0 and self.delta>0 and self.epsilon==0:
                vec_bertldaw2v = np.c_[vec_lda * self.gamma, vec_bert*self.delta]

            if self.gamma>0 and self.delta==0 and self.epsilon>0:
                vec_bertldaw2v = np.c_[vec_lda * self.gamma, vec_w2v*self.epsilon]
            
            if self.gamma==0 and self.delta>0 and self.epsilon>0:
                vec_bertldaw2v = np.c_[vec_bert*self.delta, vec_w2v*self.epsilon]
            
            if self.gamma==0 and self.delta==0 and self.epsilon>0:
                vec_bertldaw2v = vec_w2v*self.epsilon
            
            if self.gamma==0 and self.delta>0 and self.epsilon==0:
                vec_bertldaw2v = vec_bert*self.delta

            print(vec_lda.shape, vec_bert.shape, vec_w2v.shape, vec_bertldaw2v.shape)

            
            return vec_bertldaw2v
        
        elif method == "BERT_NMF_W2V":
            vec_nmf = self.vectorize(sentences, token_lists, method='NMF')
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            vec_w2v = self.vectorize(sentences, token_lists, method='W2V')
            self.nmfvec=vec_nmf
            self.bertvec=vec_bert
            self.w2vvec=vec_w2v

            vec_bertnmfw2v = np.c_[vec_nmf * self.gamma, vec_bert*self.delta, vec_w2v*self.epsilon]

            print(vec_nmf.shape, vec_bert.shape, vec_w2v.shape, vec_bertnmfw2v.shape)

            return vec_bertnmfw2v
        
        elif method == "W2V":
            print("W2V")
            vec_w2v=np.zeros((len(token_lists),300))
            for i,tl in enumerate(token_lists):
                curr_v=np.zeros(300)
                num_words=0
                for el in tl:
                    word_vec=self.modelw2v[el]
                    curr_v+=word_vec
                    num_words+=1
                curr_v=curr_v/num_words
                vec_w2v[i]=curr_v

            return vec_w2v
        
        elif method == "W2V_LDA":
            vec_lda = self.vectorize(sentences, token_lists, method='LDA')
            vec_w2v = self.vectorize(sentences, token_lists, method='W2V')
            self.ldavec=vec_lda
            self.w2vvec=vec_w2v

            if self.gamma>0 and self.epsilon>0:
                vec_ldaw2v = np.c_[vec_lda * self.gamma, vec_w2v*self.epsilon]
            
            if self.gamma==0 and self.delta>0:
                vec_ldaw2v = vec_w2v*self.epsilon
            
            print(vec_lda.shape, vec_w2v.shape, vec_ldaw2v.shape)
            
            return vec_ldaw2v
        
        elif method == "W2V_NMF":
            vec_nmf = self.vectorize(sentences, token_lists, method='NMF')
            vec_w2v = self.vectorize(sentences, token_lists, method='W2V')
            self.nmfvec=vec_nmf
            self.w2vvec=vec_w2v
            vec_nmfw2v = np.c_[vec_nmf * self.gamma, vec_w2v*self.epsilon]
            
            print(vec_nmf.shape, vec_w2v.shape, vec_nmfw2v.shape)
            
            return vec_nmfw2v
        
        elif method == "BERT_W2V":
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            vec_w2v = self.vectorize(sentences, token_lists, method='W2V')
            self.bertvec=vec_bert
            self.w2vvec=vec_w2v

            if self.delta>0 and self.epsilon>0:
                vec_bertw2v = np.c_[vec_bert*self.delta, vec_w2v*self.epsilon]
            
            if self.delta>0 and self.epsilon==0:
                vec_bertw2v = vec_bert * self.delta

            if self.delta==0 and self.epsilon>0:
                vec_bertw2v = vec_w2v*self.epsilon
            

            print(vec_bert.shape, vec_w2v.shape, vec_bertw2v.shape)

            return vec_bertw2v

    def fit(self, sentences, token_lists, method=None, m_clustering=None):

        if method is None:
            method = self.method

        if m_clustering is None:
            if self.cm=="KMeans": 
                print(f"CLUSTER MODEL: {self.cm}")
                m_clustering = KMeans
                self.cluster_model = m_clustering(n_clusters=self.k, random_state=42)
            if self.cm=="KMedoids":
                print(f"CLUSTER MODEL: {self.cm}, CLUSTER METRIC: {self.cluster_metric}")
                m_clustering = KMedoids
                self.cluster_model = m_clustering(n_clusters=self.k, metric=self.cluster_metric, random_state=42, init='k-medoids++', method='pam')

        # if not self.dictionary:
        #     self.dictionary = corpora.Dictionary(token_lists)
        #     self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]
        
        
        self.vec[method] = self.vectorize(sentences, token_lists, method)
        
        # if self.scale:
        #     print("SCALE DATA")
        #     scaler = MinMaxScaler()
        #     normalized_data=scaler.fit_transform(self.vec[method])
        #     self.vec[method]=normalized_data   
            
        if self.reduce:
            print("REDUCE DATA")
            if self.DEBUG:
                print("### DEBUG ###")
                row_mean=np.mean(self.vec[method],1)
                row_std=np.std(self.vec[method],1)
                col_mean=np.mean(self.vec[method],0)
                col_std=np.std(self.vec[method],0)
                print(f"MEAN: {np.mean(self.vec[method])}, STD: {np.std(self.vec[method])}")
                print(f"ROW - MEAN: {row_mean}")
                print(f"ROW - STD: {row_std}")
                print(f"COL - MEAN: {col_mean}")
                print(f"COL - STD: {col_std}")
                for i in range(10):
                    print(self.vec[method][i])
                
            if not self.AE:
                self.AE = Autoencoder(epochs=self.AE_epochs, latent_dim=self.AE_latent_dim)
                self.AE.fit(self.vec[method])
            vec = self.AE.encoder.predict(self.vec[method])
            self.vec[method]=vec
            
            if self.DEBUG:
                print("### DEBUG ###")
                row_mean=np.mean(vec,1)
                row_std=np.std(vec,1)
                col_mean=np.mean(vec,0)
                col_std=np.std(vec,0)
                print(f"MEAN: {np.mean(vec)}, STD: {np.std(vec)}")
                print(f"ROW - MEAN: {row_mean}")
                print(f"ROW - STD: {row_std}")
                print(f"COL - MEAN: {col_mean}")
                print(f"COL - STD: {col_std}")
                df=pd.DataFrame(self.vec[method])
                print(df)
        
        if self.DEBUG:
            print("BEFORE NORMALIZATION")
            pca = PCA(n_components=2)
            print(self.vec[method].shape)
            components = pca.fit_transform(self.vec[method])
            components=np.array(components)
            plt.scatter(components[:,0],components[:,1])
            plt.show()
                
        if self.scale:
            print("SCALE DATA")
            scaler = StandardScaler()
            normalized_data=scaler.fit_transform(self.vec[method])
            self.vec[method]=normalized_data    
                         
        if self.DEBUG:
            print("AFTER NORMALIZATION")
            pca = PCA(n_components=2)
            print(self.vec[method].shape)
            components=np.array(components)
            plt.scatter(components[:,0],components[:,1])
            plt.show()
        
        if self.DEBUG:
            print("### DEBUG ###")
            row_mean=np.mean(vec,1)
            row_std=np.std(vec,1)
            col_mean=np.mean(vec,0)
            col_std=np.std(vec,0)
            print(f"MEAN: {np.mean(vec)}, STD: {np.std(vec)}")
            print(f"ROW - MEAN: {row_mean}")
            print(f"ROW - STD: {row_std}")
            print(f"COL - MEAN: {col_mean}")
            print(f"COL - STD: {col_std}")
            df=pd.DataFrame(self.vec[method])
            print(df)
        
        print("CLUSTER INPUT DIM:",self.vec[method].shape)
        self.cluster_model.fit(self.vec[method])

    def predict(self, sentences, token_lists):
        print("PREDICT")
        other_corpus=[self.dictionary.doc2bow(text) for text in token_lists]
        self.corpus=other_corpus
        vec = self.vectorize(sentences, token_lists, self.method)
        vec = self.AE.encoder.predict(vec)
        lbs = self.cluster_model.predict(vec)
        return lbs


#### RUNNING FUNCTIONS

In [52]:
def Topic():
  print("\n")
  print("TOPIC MODEL...")
  f=open(f"PIPE/{name}/{folder}/{name}_InputData_{folder}.txt","r")
  lines=f.readlines()
  sentences=[]
  token_lists=[]
  idx_in=[]
  for i,line in enumerate(lines):
      line=line.rstrip("\n")
        #print(i,type(line),line)
      row_tk=[]
      for el in line.split(", "):
          el=el.replace("[","")
          el=el.replace("]","")
          el=el.replace("'","")
          el=el.replace(" ","_")
          row_tk.append(el)
        #print(i,type(row_tk),row_tk)
      sent=" ".join(row_tk)
        #print(i,sent,"\n")
      sentences.append(sent)
      token_lists.append(row_tk)
      idx_in.append(i)
  f.close()
  print("\tLOADED",len(sentences),"INPUT SENTENCES")

  f=open(f"PIPE/{name}/{folder}/{name}_Raw_{folder}.txt","r")
  lines=f.readlines()
  originals=[]
  for i,line in enumerate(lines):
      line=line.rstrip("\n")
      originals.append(line)
  f.close()
  print("\tLOADED",len(originals),"RAW SENTENCES")

  print("\tLOAD MODEL")

  tm=pickle.load(open(f"PIPE/INPUT/{name}_{folder}.model", "rb"))

  topics_words=tm.tw
  print(len(topics_words))

  predictions=tm.predict(sentences, token_lists)
  print(len(predictions))
  print(predictions)

  print("\tWRITE PREDICTIONS")
  f = open(f"PIPE/RESULT/{name}_{folder}_result.csv", "w")
  for idx in range(len(sentences)):
      tw = topics_words[predictions[idx]]
      tw_s = " ".join(tw)
      org = originals[idx].replace(",", " ")
      output_str = str(predictions[idx]) + ", " + tw_s + ", " + org + ", " + sentences[idx]
      print(output_str, file=f)
  f.close()
  print("... TOPIC MODEL")

## PIPELINE

In [53]:
def Main():
  Data()
  Topic()

In [54]:
Main()

	/content/gdrive/MyDrive/Colab Notebooks/PIPE/INPUT/Spark_input.log
	REMOVE POSSIBLE ACCENTED CHARACTERS
	REMOVE SPECIAL
	CAMEL CASE SPLIT
	CONCATENATION WRONG CAMEL CASE
	EXPAND POSSIBLE CONTRACTIONS
	TOKENIZATION
	REMOVE POSSIBLE STOPWORDS
	BEFORE: 3, AFTER: 3, DIFF: 0, %: 0.0%
	POS TAGGING & CHUNKING
	LEMMING
		instead NOT added since RB NOT admitted
		Do you want to change the TAG for RB (prev RB): n
		You entered: n
		Element Discarded


		There are 0 words with more than one tag
	REGEX
	CONCATENATION WRONG CAMEL CASE
	WORDS TO TAKE ONLY THEIR BIGRAM/TRIGRAM INSIDE LOG ROWS: ['manager', 'executor', 'storage', 'block']
	MAKE NGRAMS


TOPIC MODEL...
	LOADED 3 INPUT SENTENCES
	LOADED 3 RAW SENTENCES
	LOAD MODEL
39
PREDICT
	MAKE CORPUS
NMF
	LEN CORPUS 3
BERT
W2V
(3, 39) (3, 768) (3, 300) (3, 1107)
3
[25  1  6]
	WRITE PREDICTIONS
... TOPIC MODEL
